In [93]:
import pandas as pd
import numpy as np

from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time
import sys
import gc
import pickle
sys.version_info

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [94]:
test = pd.read_csv('test_data.csv').drop(columns = ['Unnamed: 0'])
test.shape

(375232, 3)

In [95]:
# load data
df = pd.read_csv('data_weekly_loc_dep.csv').drop(columns = ['Unnamed: 0'])

In [96]:
store_df = pd.read_csv('store_lookup.txt', sep = '|')
store_df = store_df[['LOC_ID', 'DISTRICT', 'MARKET']]

df = df.merge(store_df, on = 'LOC_ID', how = 'left')

In [97]:
df = df.groupby(['FISCAL_WEEK_ID', 'DEPARTMENT_ID', 'DISTRICT']).agg(
            {
                "GROSS_SALES": "sum",
                "PROMO_DISCOUNT": "sum",
                "CLEARANCE_DISCOUNT": "sum",
                "COUPON_DISCOUNT": "sum",
                "MISCELLANOUS_DISCOUNT": "sum",
                "ACTUAL_SALES": "sum",
                "UNITS_SOLD": "sum",
                "INITIAL_MARGIN": "sum",
            }
        ).reset_index()

In [98]:
store_df = store_df.groupby(['DISTRICT', 'MARKET'])['LOC_ID'].count().reset_index()
# df = df.merge(store_df, on = 'DISTRICT', how = 'left')

In [99]:
df = df.merge(store_df[['DISTRICT', 'MARKET']], on = 'DISTRICT', how = 'left')

In [100]:
# add week_num
calendar_df = pd.read_csv('fiscal_calendar.txt', sep='|')
week_num_df = calendar_df.groupby('FISCAL_WEEK_ID')['DAY_ID'].count().reset_index().reset_index().rename(columns = {'index':'Week_num'})
week_num_df = week_num_df[['FISCAL_WEEK_ID', 'Week_num']]
df = df.merge(week_num_df, on = 'FISCAL_WEEK_ID', how = 'left')

In [101]:
department_df = pd.read_csv('dept_lookup.txt', sep = '|')
df = df.merge(department_df, on = 'DEPARTMENT_ID', how = 'left')

In [102]:
df['department_code'] =  LabelEncoder().fit_transform(df['DEPARTMENT'])
df['shop_code'] = LabelEncoder().fit_transform(df['SHOP'])

In [103]:
df['UNIT_PRICE'] = df['ACTUAL_SALES'] / df['UNITS_SOLD']

In [104]:
# combine test and df
ts = time.time()
df = pd.concat([df, test], ignore_index=True, sort=False, keys=['Week_num', 'DEPARTMENT_ID', 'DISTRICT'])
df.fillna(0, inplace=True) 
time.time() - ts

1.4666812419891357

In [105]:
# df.to_csv('result.csv', index = False)

In [12]:
# df = pd.read_csv('result.csv')

In [13]:
# check 0 unit price
# df[(df['UNIT_PRICE'] == 0) & (df['UNITS_SOLD'] >0)]

In [106]:
def lag_feature(df, lags, col):
    tmp = df[['Week_num','DEPARTMENT_ID','DISTRICT',col]]
    for i in tqdm(lags):
        shifted = tmp.copy()
        shifted.columns = ['Week_num','DEPARTMENT_ID','DISTRICT', col+'_lag_'+str(i)]
        shifted['Week_num'] += i
        df = pd.merge(df, shifted, on=['Week_num','DEPARTMENT_ID','DISTRICT'], how='left')
    return df

In [107]:
ts = time.time()
df = lag_feature(df, [4,8,12,26,52], 'ACTUAL_SALES')
time.time() - ts

100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


5.199804067611694

In [108]:
def add_group_stats(matrix_, groupby_feats, target, enc_feat, last_periods):
    if not 'Week_num' in groupby_feats:
        print ('Week_num must in groupby_feats')
        return matrix_
    
    group = matrix_.groupby(groupby_feats)[target].sum().reset_index()
    max_lags = np.max(last_periods)
    for i in range(1,max_lags+1):
        shifted = group[groupby_feats+[target]].copy(deep=True)
        shifted['Week_num'] += i
        shifted.rename({target:target+'_lag_'+str(i)},axis=1,inplace=True)
        group = group.merge(shifted, on=groupby_feats, how='left')
    group.fillna(0,inplace=True)
    for period in last_periods:
        lag_feats = [target+'_lag_'+str(lag) for lag in np.arange(1,period+1)]
        # we do not use mean and svd directly because we want to include months with sales = 0
        mean = group[lag_feats].sum(axis=1)/float(period)
        mean2 = (group[lag_feats]**2).sum(axis=1)/float(period)
        group[enc_feat+'_avg_sale_last_'+str(period)] = mean
        group[enc_feat+'_std_sale_last_'+str(period)] = (mean2 - mean**2).apply(np.sqrt)
        group[enc_feat+'_std_sale_last_'+str(period)].replace(np.inf,0,inplace=True)
        # divide by mean, this scales the features for NN
        group[enc_feat+'_avg_sale_last_'+str(period)] /= group[enc_feat+'_avg_sale_last_'+str(period)].mean()
        group[enc_feat+'_std_sale_last_'+str(period)] /= group[enc_feat+'_std_sale_last_'+str(period)].mean()
    cols = groupby_feats + [f_ for f_ in group.columns.values if f_.find('_sale_last_')>=0]
    matrix = matrix_.merge(group[cols], on=groupby_feats, how='left')
    return matrix

In [109]:
ts = time.time()
df = add_group_stats(df, ['Week_num', 'DEPARTMENT_ID'], 'ACTUAL_SALES', 'department', [26,52])
df = add_group_stats(df, ['Week_num', 'DISTRICT'], 'ACTUAL_SALES', 'district', [26,52])
df = add_group_stats(df, ['Week_num', 'MARKET'], 'ACTUAL_SALES', 'market', [52])
df = add_group_stats(df, ['Week_num', 'department_code'], 'ACTUAL_SALES', 'dep', [52])
df = add_group_stats(df, ['Week_num', 'shop_code'], 'ACTUAL_SALES', 'shop', [52])

In [110]:
#first use target encoding each group, then shift month to creat lag features
def target_encoding(matrix_, groupby_feats, target, enc_feat, lags):
    print ('target encoding for',groupby_feats)
    group = matrix_.groupby(groupby_feats).agg({target:'mean'})
    group.columns = [enc_feat]
    group.reset_index(inplace=True)
    matrix = matrix_.merge(group, on=groupby_feats, how='left')
    matrix[enc_feat] = matrix[enc_feat].astype(np.float16)
    matrix = lag_feature(matrix, lags, enc_feat)
    matrix.drop(enc_feat, axis=1, inplace=True)
    return matrix

In [111]:
ts = time.time()
df = target_encoding(df, ['Week_num'], 'ACTUAL_SALES', 'week_avg_actual_sales', [4])
df = target_encoding(df, ['Week_num', 'DEPARTMENT_ID'], 'ACTUAL_SALES', 'week_dept_avg_actual_sales', [4,8,12,26,52])
df = target_encoding(df, ['Week_num', 'DISTRICT'], 'ACTUAL_SALES', 'week_district_avg_actual_sales', [4,8,12,26,52])
df = target_encoding(df, ['Week_num', 'MARKET'], 'ACTUAL_SALES', 'week_market_avg_actual_sales', [4])
df = target_encoding(df, ['Week_num', 'DEPARTMENT_ID', 'MARKET'], 'ACTUAL_SALES', 'week_department_market_avg_actual_sales', [4])
time.time() - ts

target encoding for ['Week_num']


100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


target encoding for ['Week_num', 'DEPARTMENT_ID']


100%|██████████| 5/5 [00:05<00:00,  1.06s/it]


target encoding for ['Week_num', 'DISTRICT']


...

target encoding for ['Week_num', 'MARKET']


100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


target encoding for ['Week_num', 'DEPARTMENT_ID', 'MARKET']


100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


29.886138677597046

In [112]:
ts = time.time()
group = df.groupby(['DEPARTMENT_ID']).agg({'UNIT_PRICE': ['mean']})
group.columns = ['department_avg_unit_price']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['DEPARTMENT_ID'], how='left')
df['department_avg_unit_price'] = df['department_avg_unit_price'].astype(np.float16)

group = df.groupby(['Week_num','DEPARTMENT_ID']).agg({'UNIT_PRICE': ['mean']})
group.columns = ['week_department_avg_unit_price']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['Week_num','DEPARTMENT_ID'], how='left')
df['week_department_avg_unit_price'] = df['week_department_avg_unit_price'].astype(np.float16)

lags = [4,8,12,16,20,26]
df= lag_feature(df, lags, 'week_department_avg_unit_price')

for i in lags:
    df['delta_price_lag_'+str(i)] = \
        (df['week_department_avg_unit_price_lag_'+str(i)] - df['department_avg_unit_price']) / df['department_avg_unit_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
df['delta_price_lag'] = df.apply(select_trend, axis=1)
df['delta_price_lag'] = df['delta_price_lag'].astype(np.float16)
df['delta_price_lag'].fillna(0, inplace=True)

# https://stackoverflow.com/questions/31828240/first-non-null-value-per-row-from-a-list-of-pandas-columns/31828559
# matrix['price_trend'] = matrix[['delta_price_lag_1','delta_price_lag_2','delta_price_lag_3']].bfill(axis=1).iloc[:, 0]
# Invalid dtype for backfill_2d [float16]

fetures_to_drop = ['department_avg_unit_price', 'week_department_avg_unit_price']
for i in lags:
    fetures_to_drop += ['week_department_avg_unit_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

df.drop(fetures_to_drop, axis=1, inplace=True)

time.time() - ts

100%|██████████| 6/6 [00:07<00:00,  1.23s/it]


44.603397607803345

In [113]:
ts = time.time()
group = df.groupby(['Week_num','DISTRICT']).agg({'ACTUAL_SALES': ['sum']})
group.columns = ['week_district_actual_sales']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['Week_num','DISTRICT'], how='left')
df['week_district_actual_sales'] = df['week_district_actual_sales'].astype(np.float32)

group = group.groupby(['DISTRICT']).agg({'week_district_actual_sales': ['mean']})
group.columns = ['district_avg_actual_sales']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=['DISTRICT'], how='left')
df['district_avg_actual_sales'] = df['district_avg_actual_sales'].astype(np.float32)

df['delta_actual_sales'] = (df['week_district_actual_sales'] - df['district_avg_actual_sales']) / df['district_avg_actual_sales']
df['delta_actual_sales'] = df['delta_actual_sales'].astype(np.float16)

df = lag_feature(df, [4], 'delta_actual_sales')

df.drop(['week_district_actual_sales','district_avg_actual_sales','delta_actual_sales'], axis=1, inplace=True)
time.time() - ts

100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


5.861367464065552

In [114]:
calendar_df = pd.read_csv('fiscal_calendar.txt', sep='|')
week_num_df = calendar_df.groupby(['FISCAL_WEEK_ID','FISCAL_MONTH_ID'])['DAY_ID'].count().reset_index().reset_index().rename(columns = {'index':'Week_num'})
week_num_test = week_num_df.copy()
week_num_test['Week_num'] = week_num_test['Week_num'].astype(int) + 52
week_num_df = pd.concat([week_num_df, week_num_test.iloc[-52:,]]).reset_index(drop = True)
# # week_num_df = week_num_df[['FISCAL_MONTH_ID', 'month_num']]
# # df = df.merge(week_num_df, on = 'FISCAL_MONTH_ID', how = 'left')
week_num_df["Month_num"] = week_num_df["FISCAL_MONTH_ID"].apply(
    lambda x: str(x)[-2:]
)
df = df.merge(week_num_df[['Week_num', 'Month_num']], on = 'Week_num', how = 'left')

In [115]:
df['year'] = (df['Week_num'] / 52).astype(np.int8)

In [116]:
#Month since last sale for each shop/item pair.
ts = time.time()
last_sale = pd.DataFrame()
for week in range(1,165):    
    last_month = df.loc[(df['Week_num']<week)&(df['UNITS_SOLD']>0)].groupby(['DEPARTMENT_ID','DISTRICT'])['Week_num'].max()
    new_df = pd.DataFrame({'Week_num':np.ones([last_month.shape[0],])*week,
                       'DEPARTMENT_ID': last_month.index.get_level_values(0).values,
                       'DISTRICT': last_month.index.get_level_values(1).values,
                       'department_district_last_sale': last_month.values})
    last_sale = last_sale.append(new_df)
last_sale['Week_num'] = last_sale['Week_num'].astype(np.int8)

df = df.merge(last_sale, on=['Week_num','DEPARTMENT_ID','DISTRICT'], how='left')
time.time() - ts

30.898027420043945

In [117]:
#Month since last sale for each item.
ts = time.time()
last_sale = pd.DataFrame()
for week in range(1,165):    
    last_month = df.loc[(df['Week_num']<week)&(df['UNITS_SOLD']>0)].groupby('DEPARTMENT_ID')['Week_num'].max()
    new_df = pd.DataFrame({'Week_num':np.ones([last_month.shape[0],])*week,
                       'DEPARTMENT_ID': last_month.index.values,
                       'department_last_sale': last_month.values})
    last_sale = last_sale.append(new_df)
last_sale['Week_num'] = last_sale['Week_num'].astype(np.int8)

df = df.merge(last_sale, on=['Week_num','DEPARTMENT_ID'], how='left')
time.time() - ts

24.068939447402954

In [118]:
# Months since the first sale for each shop/item pair and for item only.
ts = time.time()
df['department_district_first_sale'] = df['Week_num'] - df.groupby(['DEPARTMENT_ID','DISTRICT'])['Week_num'].transform('min')
df['department_first_sale'] = df['Week_num'] - df.groupby('DEPARTMENT_ID')['Week_num'].transform('min')
time.time() - ts

0.23488211631774902

In [119]:
df = df[df.Week_num > 51]

In [120]:
df[df['Month_num'].isna()]

,FISCAL_WEEK_ID,DEPARTMENT_ID,DISTRICT,GROSS_SALES,PROMO_DISCOUNT,CLEARANCE_DISCOUNT,COUPON_DISCOUNT,MISCELLANOUS_DISCOUNT,ACTUAL_SALES,UNITS_SOLD,...,week_market_avg_actual_sales_lag_4,week_department_market_avg_actual_sales_lag_4,delta_price_lag,delta_actual_sales_lag_4,Month_num,year,department_district_last_sale,department_last_sale,department_district_first_sale,department_first_sale


In [121]:
df.isna().sum()

FISCAL_WEEK_ID                                        0
DEPARTMENT_ID                                         0
DISTRICT                                              0
GROSS_SALES                                           0
PROMO_DISCOUNT                                        0
CLEARANCE_DISCOUNT                                    0
COUPON_DISCOUNT                                       0
MISCELLANOUS_DISCOUNT                                 0
ACTUAL_SALES                                          0
UNITS_SOLD                                            0
INITIAL_MARGIN                                        0
MARKET                                                0
Week_num                                              0
DEPARTMENT                                            0
SHOP                                                  0
SBU                                                   0
department_code                                       0
shop_code                                       

In [123]:
df.to_csv('new_data_for_model.csv', index = False)